In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ai_mathematical_olympiad_progress_prize_3_path = kagglehub.competition_download('ai-mathematical-olympiad-progress-prize-3')
mccocoful_latest_mdc_whls_path = kagglehub.notebook_output_download('mccocoful/latest-mdc-whls')
qwen_lm_qwq_32b_transformers_qwq_32b_awq_1_path = kagglehub.model_download('qwen-lm/qwq-32b/Transformers/qwq-32b-awq/1')
qwen_lm_qwen_3_transformers_32b_awq_1_path = kagglehub.model_download('qwen-lm/qwen-3/Transformers/32b-awq/1')

print('Data source import complete.')


In [ ]:
! uv pip uninstall --system 'tensorflow'
! uv pip install --system --no-index --find-links='/kaggle/input/latest-mdc-whls/whls' 'vllm' 'triton' 'numpy<2'

Using Python 3.11.13 environment at: /usr
Uninstalled 1 package in 3.29s
 - tensorflow==2.18.0
Using Python 3.11.13 environment at: /usr
Resolved 155 packages in 435ms                                       
Prepared 52 packages in 10.07s                                           
Uninstalled 25 packages in 359ms
Installed 52 packages in 56ms                               
 + airportsdata==20250622
 + astor==0.8.1
 - click==8.3.0
 + click==8.2.1
 + compressed-tensors==0.9.3
 + depyf==0.18.0
 + diskcache==5.6.3
 - fastapi==0.116.1
 + fastapi==0.115.14
 + fastapi-cli==0.0.7
 + gguf==0.17.1
 + httptools==0.6.4
 - importlib-metadata==8.7.0
 + importlib-metadata==8.0.0
 + interegular==0.3.3
 - lark==1.3.1
 + lark==1.2.2
 + llguidance==0.7.30
 - llvmlite==0.43.0
 + llvmlite==0.44.0
 + lm-format-enforcer==0.10.11
 + mistral-common==1.6.2
 + msgspec==0.19.0
 - numba==0.60.0
 + numba==0.61.2
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.4.5.8
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvi

In [ ]:
from transformers import set_seed
set_seed(2024)


In [ ]:
import os
import gc
import time
import warnings

import pandas as pd
import polars as pl

import torch
import kaggle_evaluation.aimo_3_inference_server

pd.set_option('display.max_colwidth', None)
cutoff_time = time.time() + (4 * 60 + 45) * 60

In [ ]:
from vllm import LLM, SamplingParams

warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

llm_model_pth = '/kaggle/input/qwen-3/transformers/32b-awq/1'

llm = LLM(
    llm_model_pth,
    #dtype="half",                -> Changed this
    #max_num_seqs=128,            -> Changed this
    max_model_len=32768,#4096*10,
    trust_remote_code=True,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.96,
)

INFO 12-02 18:43:37 [__init__.py:239] Automatically detected platform cuda.
INFO 12-02 18:43:54 [config.py:717] This model supports multiple tasks: {'score', 'embed', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 12-02 18:43:55 [awq_marlin.py:113] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 12-02 18:43:55 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 12-02 18:43:56 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 12-02 18:44:00 [__init__.py:239] Automatically detected platform cuda.
INFO 12-02 18:44:04 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='/kaggle/input/qwen-3/transformers/32b-awq/1', speculative_config=None, tokenizer='/kaggle/

[W1202 18:44:05.413660097 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1202 18:44:05.414298338 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-02 18:44:05 [parallel_state.py:1004] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 12-02 18:44:05 [cuda.py:221] Using Flash Attention backend on V1 engine.
WARNING 12-02 18:44:05 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 12-02 18:44:05 [gpu_model_runner.py:1329] Starting to load model /kaggle/input/qwen-3/transformers/32b-awq/1...


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:42<02:06, 42.21s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [01:13<01:11, 35.58s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [01:40<00:31, 31.61s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:22<00:00, 36.04s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:22<00:00, 35.72s/it]



INFO 12-02 18:46:28 [loader.py:458] Loading weights took 142.95 seconds
INFO 12-02 18:46:30 [gpu_model_runner.py:1347] Model loading took 18.1453 GiB and 144.989844 seconds
INFO 12-02 18:46:52 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/922129f9e7/rank_0_0 for vLLM's torch.compile
INFO 12-02 18:46:52 [backends.py:430] Dynamo bytecode transform time: 21.90 s
INFO 12-02 18:46:57 [backends.py:136] Cache the graph of shape None for later use
INFO 12-02 18:47:50 [backends.py:148] Compiling a graph for general shape takes 56.45 s
INFO 12-02 18:49:11 [monitor.py:33] torch.compile takes 78.35 s in total
INFO 12-02 18:49:13 [kv_cache_utils.py:634] GPU KV cache size: 211,376 tokens
INFO 12-02 18:49:13 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 6.45x
INFO 12-02 18:49:56 [gpu_model_runner.py:1686] Graph capturing finished in 43 secs, took 1.40 GiB
INFO 12-02 18:49:56 [core.py:159] init engine (profile, create kv cache, warmup model

In [ ]:
tokenizer = llm.get_tokenizer()


In [ ]:
import re

#prompts
thoughts = [

    'Please use chained reasoning to put the answer in \\boxed{}.',
    'Please reflect and verify while reasoning and put the answer in \\boxed{}.',
    'Solve the following problem using concise and clear reasoning by placing the answer in \\boxed{}.',
    'You are a helpful and reflective maths assistant, please reason step by step to put the answer in \\boxed{}.',
    'You are the smartest maths expert in the world, please spike this question and put the answer in \\boxed{}.'
]

#create single prompt
def make_next_prompt(text,round_idx):
    default_prompt = thoughts[(round_idx+1)%len(thoughts)]
    default_python_code = f"print('{default_prompt}')"
    return default_python_code

#extract python code from response
def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        ans = "\n\n".join(matches)
        #print(f'Extracted python code: {ans}')
        return ans
    return ""

#extract all code segments
def extract_python_code_list(text):
    pattern = r'```python\s*(.*?)\s*```'
    ans=[]
    matches = re.findall(pattern, text, re.DOTALL)
    for m in matches:
        ans.append(m)
    return ans

#process the code
def process_python_code(query):
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        new_rows.append(row)
    ans = "\n".join(new_rows)
    print(f'Processed python code: {ans}')
    return ans

import re

#extract the answer from the boxes
def extract_boxed_texts(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return []
    ans = []
    for content in matches:
        if content.isdigit():
            num = int(content)
        else:
            nums = re.findall(r'\d+', content)
            if not nums:
                continue
            num = int(nums[-1])
        ans.append(num % 100000)
    return ans

#extract the integer answer modulo 100000 from the boxes
def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return -1
    content = matches[0]
    if content.isdigit():
        num = int(content)
    else:
        nums = re.findall(r'\d+', content)
        if not nums:
            return -1
        num = int(nums[-1])
    return num % 100000

#select the final answer based on the frequency/ majoity voting
from collections import Counter
def select_answer(answers):
    valid_answers = []
    for answer in answers:
        try:
            if int(answer) == float(answer):
                valid_answers.append(int(answer)%100000)
        except:
            pass
    if not valid_answers:
        return 49
    _, answer = sorted([(v,k) for k,v in Counter(valid_answers).items()], reverse=True)[0]
    return answer

In [ ]:
import os
import tempfile
import subprocess

#Python REPL to execute code. taken from NuminaMath Solution
class PythonREPL:
    def __init__(self, timeout=8):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)

            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            stdout = result.stdout.strip()
            stderr = result.stderr.strip()

            if result.returncode == 0:
                return True, stdout
            else:
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = stderr.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                # Include stdout in the error case
                combined_output = f"{stdout}\n{cleaned_error_msg}" if stdout else cleaned_error_msg
                return False, combined_output


In [ ]:
list_of_texts = [
    tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True
    )
    for messages in [[{"role": "user", "content": "hi"}]]
]

In [ ]:
#define the sampling parameters
sampling_params = SamplingParams(
    temperature=0.95,              # Controls randomness in generation: higher values (e.g., 1.0) produce more diverse output.
    min_p=0.01,                   # Minimum cumulative probability for nucleus sampling, filtering out unlikely tokens.
    skip_special_tokens=True,
    # max_tokens=1800,
    max_tokens=32768,             # Sets a very high limit for token generation to handle longer outputs.
    # stop=["```output"],
)

#generate prompts in batch
def batch_message_generate(list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]

    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )

    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        print(messages[-1])

    return list_of_messages


In [ ]:
def batch_message_filter(list_of_messages,list_of_idx) -> tuple[list[list[dict]], list[str]]:
    global answer_contributions
    extracted_answers = []
    list_of_messages_to_keep = []
    list_of_idx_to_keep = []
    for idx,messages in zip(list_of_idx,list_of_messages):
        answers = extract_boxed_texts(messages[-1]['content'])
        if answers:
            extracted_answers.extend(answers)
            for answer in answers:
                answer_contributions[answer].append(idx)
        else:
            list_of_messages_to_keep.append(messages)
            list_of_idx_to_keep.append(idx)
    return list_of_messages_to_keep, extracted_answers, list_of_idx_to_keep

In [ ]:
#execute the codes in the responses
def batch_message_execute(list_of_messages,round_idx) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'],round_idx)
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
        except Exception as e:
            output = str(e)
        messages.append({'role': 'user', 'content': output})
        print(messages[-1])
    return list_of_messages

#execute the code and generate the answer from responses
def batch_message_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
            if success:
                patten = r'(\d+)'
                matches = re.findall(patten, output)
                if matches:
                    for match in matches:
                        ans.append(int(match)%100000)
                        ans.append(int(match)%100000) #代码权重高于自然语言，所以添加两次
        except Exception as e:
            output = str(e)
        print(f'python code output: {output}')
    return ans

#execute code and generate answer for all elements in batch
def batch_message_list_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code_list = extract_python_code_list(messages[-1]['content'])
        for python_code in python_code_list:
            python_code = process_python_code(python_code)
            try:
                success, output = PythonREPL()(python_code)
                if success:
                    patten = r'(\d+)'
                    matches = re.findall(patten, output)
                    if matches:
                        for match in matches:
                            ans.append(int(match)%100000)
                            ans.append(int(match)%100000)
            except Exception as e:
                output = str(e)
            print(f'python code output: {output}')
    return ans


In [ ]:
import os

import pandas as pd
import polars as pl

#API for competition submission
import kaggle_evaluation.aimo_3_inference_server

In [ ]:
#correct answers for reference problems
def get_correct_answer(question):
    if 'Let $ABC$ be an acute-angled triangle with integer side' in question: return 336
    if 'Define a function $f \colon \mathbb{Z}_{\geq 1} \to \mathbb{Z}_{\geq 1}$' in question: return 32951
    if 'A tournament is held with $2^{20}$ runners each of which' in question: return 21818
    if 'On a blackboard, Ken starts off by' in question: return 32193
    if 'Let $ABC$ be a triangle with $AB \neq AC$, circumcircle' in question: return 57447
    if 'Let $n \geq 6$ be a positive integer' in question: return 8687
    if 'Alice and Bob are each holding some integer number of sweets' in question: return 50
    if 'Let $f \colon \mathbb{Z}_{\geq 1} \to \mathbb{Z}_{\geq 1}$ be a function' in question: return 580
    if 'A $500 \times 500$ square is divided into $k$ rectangles' in question: return 520
    if 'Let $\mathcal{F}$ be the set of functions $\alpha \colon \mathbb{Z}\to' in question: return 160
    return 0

In [ ]:
from collections import Counter, defaultdict
g_score = 0
g_count = 0
prompt_score = Counter()
answer_contributions = defaultdict(list)
def predict_for_question(question: str) -> int:
    global g_score
    global g_count
    global prompt_score
    global answer_contributions
    question += "\nIf the final answer is a number larger than 100000, take modulo 100000. "
    if time.time() > cutoff_time:
        return 210
    print(question)

    list_of_messages = [
        [
            {"role": "system", "content": thoughts[k]},
            {"role": "user", "content": question}
        ] for k in range(5)
    ]

    all_extracted_answers = []
    list_of_idx = list(range(len(list_of_messages)))
    max_round = 1
    for round_idx in range(max_round):
        print(f"round {round_idx+1}")
        list_of_messages = batch_message_generate(list_of_messages)
        #extracted_python_answer = batch_message_execute_and_get_answer(list_of_messages,round_idx)
        extracted_python_answer = batch_message_list_execute_and_get_answer(list_of_messages,round_idx)
        list_of_messages, extracted_answers, list_of_idx  = batch_message_filter(list_of_messages, list_of_idx)
        all_extracted_answers.extend(extracted_python_answer)
        all_extracted_answers.extend(extracted_answers)
        print("extracted boxed answers:",extracted_answers)
        print("extracted python answers:",extracted_python_answer)
        print("all extracted answers:",all_extracted_answers)
        if not list_of_messages:
            break
        #list_of_messages = batch_message_execute(list_of_messages,round_idx)
    answer = select_answer(all_extracted_answers)
    print("answer:",answer)
    correct_answer = get_correct_answer(question)
    print("correct answer:",correct_answer)
    g_count += 1
    if str(answer) == str(correct_answer):
        g_score += 1

    print(f"score: {g_score}/{g_count}")
    print("\n\n")
    return answer

In [ ]:
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)

    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [ ]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [ ]:
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'reference.csv',
        )
    )

------
86e8e5
Let $n \geq 6$ be a positive integer. We call a positive integer $n$-Norwegian if it has three distinct positive divisors whose sum is equal to $n$. Let $f(n)$ denote the smallest $n$-Norwegian positive integer. Let $M=3^{2025!}$ and for a non-negative integer $c$ define 
\begin{equation*}
    g(c)=\frac{1}{2025!}\left\lfloor \frac{2025! f(M+c)}{M}\right\rfloor.
\end{equation*}
We can write 
\begin{equation*}
    g(0)+g(4M)+g(1848374)+g(10162574)+g(265710644)+g(44636594)=\frac{p}{q}
\end{equation*}
where $p$ and $q$ are coprime positive integers. What is the remainder when $p+q$ is divided by $99991$?
If the final answer is a number larger than 100000, take modulo 100000. 
round 1


Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
import pandas as pd

try:
    df = pd.read_parquet('submission.parquet').sort_values('id')
    print(df)
except Exception:
    print('No file generated!')